# Lab 3: Effects of Strain on the Compressive Viscoelastic Properties of Articular Cartilage

## Goals of the Lab Report

- **Hypotheses** - What are the effects of the experimental variable (compressive
  strain) on:
  - Instantaneous modulus, $E_{0}$
  - Equilibrium Modulus, $E_{eq}$
  - Time Constant, $\tau$

- Present the $\text{mean} \pm \text{standard deviation}$ and perform statistical 
  tests for:
  - Geometric Dimensions: Diameter, Thickness
  - Contact Area (Cross-Sectional Area), $A$
  - Equilibrium Modulus, $E_{eq}$
  - Instantaneous Modulus, $E_{0}$
  - Time Constant, $\tau$
  - Stretch Parameter, $\beta$
  - % Relaxation

## Data Preprocessing and Analysis

- Truncate the data to only include the stress relaxation portion (peak force
  onwards)
- Flip the data (make the force / stress positive)
- Convert force into compressive normal stress
- Fit the data with a **stretched exponential model** of stress relaxation
  - Polydisperse polymer reptation model
    - $\sigma(t) = \left(E_0\varepsilon_0 - E_{eq}\varepsilon_0\right)\exp\left(-\left(\frac{t}{\tau}\right)^\beta\right) + E_{eq}\varepsilon_0$
    - $\sigma(t) = \varepsilon_0 \left[ (E_0 - E_{eq})\exp\left(-\left(\frac{t}{\tau}\right)^\beta\right) + E_{eq} \right]$
    - Parameters:
      - $E_0$: Instantaneous modulus
      - $E_{eq}$: Equilibrium modulus
      - $\tau$: Time constant
      - $\beta$: Stretch parameter
    - Constants:
      - $\varepsilon_0$: Constant strain
  - % Relaxation
    - $=100\% \cdot \left(1 - \frac{E_{eq}}{E_0}\right)$

In [7]:
# Google Colab: Run this to mount your Google Drive
# from google.colab import drive
# import os
# drive.mount('/content/gdrive', force_remount=True)
# os.chdir('/content/gdrive/MyDrive/BMES 301/Lab 1')

# For local only
import os; os.chdir("/home/kabil/sietch/courses/bmes301/lab3")

In [8]:
# Notebook setup

%load_ext autoreload
%autoreload 2

## Import necessary libraries
### Data analysis
import pandas as pd
import numpy as np
import tools

### Plotting
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns

### Setup
from pathlib import Path
import re

## Define data path
datapath = Path("data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [9]:
# Load data

## Regular expression to pull team information from file name
meta = re.compile(r"g(?P<team>\d{2})_(?P<strain_level>\w*)")

## Load all data into a list
data = []
for file in datapath.glob("lab3*.csv"):

    ## Extract sample and team information from file name
    match = meta.search(file.name)

    ## Load data
    try:
        _df = tools.load_data(file, datastart=45)
        _df['team'] = match.group('team')
        _df['strain_level'] = match.group('strain_level')
        data.append(_df)
    except AssertionError as e:
        print(f"Error loading {file.name}: {e}")

## Concatenate all data into a single dataframe
raw_data = (pd.concat(data)
    .sort_values(['team', 'time'])
    .reset_index(drop=True))
display(raw_data.head())

## Load specimen dimensions
specimen = pd.read_excel(datapath / 'lab3_specimen_dimensions.xlsx')
## Calculate area (Assuming circular cross-section)
specimen['area'] = np.pi * (specimen['Diameter (mm)'] / 2)**2

,time,disp,load,team,strain_level
0,0.0000,0.475169,-0.03,01,high
1,0.0000,-0.210118,-0.06,01,low
2,0.1464,0.463991,-0.08,01,high
3,0.1464,-0.219605,-0.12,01,low
4,0.2928,0.452410,-0.16,01,high


## Preprocessing

In [10]:
# Create figure for overlaying data segmentation
fig_segment = plt.figure(constrained_layout=True, figsize=(8, 55))
fig_segment.suptitle("Segmentation of Stress-Relaxation Portion of Data", fontsize=18)
subfigs_segment = fig_segment.subfigures(nrows=10, ncols=1)

# Create figure for stress-relaxation curves
fig_relax = plt.figure(constrained_layout=True, figsize=(6, 30))
fig_relax.suptitle("Stress-Relaxation Curves", fontsize=18)
subfigs_relax = fig_relax.subfigures(nrows=10, ncols=1)

# List to store stress-relaxation data
data = []

for team, subfig_segment, subfig_relax in zip(raw_data['team'].unique(), 
                                              subfigs_segment,
                                              subfigs_relax):

    ## Create grid for segmentation plot
    subfig_segment.suptitle(f"Team {team}", fontsize=16)
    subfig_segment.set_facecolor('#f0f0f0')
    grid_segment = subfig_segment.subplots(nrows=2, ncols=2)

    ## Create grid for relaxation plot
    subfig_relax.suptitle(f"Team {team}", fontsize=16)
    subfig_relax.set_facecolor('#f0f0f0')
    grid_relax = subfig_relax.subplots(nrows=1, ncols=2)


    for strain_level, axes_segment, axes_relax in zip(['low', 'high'], grid_segment, grid_relax):

        ## Select data for a single team and strain level
        test = (raw_data
            .query(f"strain_level == '{strain_level}' and team == '{team}'")
            .reset_index(drop=True)
            .copy())

        ## Compute compressive stress
        strain = 0.3 if strain_level == 'high' else 0.1
        J = (specimen['Group #'] == int(team)) & (specimen['Strain'] == strain)
        test['stress'] = -test['load'] / specimen.loc[J, 'area'].iloc[0]  # MPa

        ## Identify the stress-relaxation portion of the data
        I = np.argmin(test['load'])
        _data = test.iloc[I:].reset_index(drop=True).copy()

        ## Zero the time
        _data['time'] -= _data['time'].iloc[0]

        ## Plot the segmented data        
        sns.lineplot(data=test, x='time', y='load', ax=axes_segment[0])
        sns.lineplot(data=_data, x='time', y='load', ax=axes_segment[0], 
                     color='r', linewidth=7, alpha=0.3)
        axes_segment[0].plot(test['time'].iloc[I], test['load'].iloc[I], 'ro')
        axes_segment[0].set_title(f"Load Curve\n{strain_level} strain")
        sns.despine()
        
        sns.lineplot(data=test, x='time', y='disp', ax=axes_segment[1])
        sns.lineplot(data=_data, x='time', y='disp', ax=axes_segment[1], 
                     color='r', linewidth=7, alpha=0.3)
        axes_segment[1].set_title(f"Displacement Curve\n{strain_level} strain") 
        axes_segment[1].plot(test['time'].iloc[I], test['disp'].iloc[I], 'ro')
        sns.despine()

        ## Plot the stress-relaxation curves
        sns.lineplot(data=_data, x='time', y='stress', ax=axes_relax)
        axes_relax.set_xlabel("Time (s)")
        axes_relax.set_ylabel("Stress (MPa)")
        axes_relax.set_title(f"{strain_level} strain")

        ## Store the data
        _data['strain'] = 0.3 if strain_level == 'high' else 0.1
        data.append( _data[['team', 'time', 'stress', 'strain']] )

# Save segmentation figure
pdf = PdfPages("results/segmentation.pdf")
pdf.savefig(fig_segment)
pdf.close()

# Save stress-relaxation figure
pdf = PdfPages("results/stress_relaxation_curves.pdf")
pdf.savefig(fig_relax)
pdf.close()

# Close all figures
plt.close('all')

# Save stress-relaxation data
(pd.concat(data)
    .to_csv("results/stress_relaxation_data.csv", index=False))

In [11]:
mean = (specimen.groupby('Strain')
    [['Diameter (mm)', 'Thickness (mm)', 'area']].mean()
    .to_dict(orient='index'))
std = (specimen.groupby('Strain')
    [['Diameter (mm)', 'Thickness (mm)', 'area']].std()
    .to_dict(orient='index'))

In [12]:
for strain in [0.1, 0.3]:
    print(f"""
Strain: {strain} ({'High' if strain == 0.3 else 'Low'})
Diameter (mm): {mean[strain]['Diameter (mm)']:.2f} +/- {std[strain]['Diameter (mm)']:.2f}
Thickness (mm): {mean[strain]['Thickness (mm)']:.2f} +/- {std[strain]['Thickness (mm)']:.2f}
Area (mm^2): {mean[strain]['area']:.2f} +/- {std[strain]['area']:.2f}
    """)


Strain: 0.1 (Low)
Diameter (mm): 3.98 +/- 0.29
Thickness (mm): 1.19 +/- 0.42
Area (mm^2): 12.50 +/- 1.84
    

Strain: 0.3 (High)
Diameter (mm): 4.03 +/- 0.25
Thickness (mm): 1.34 +/- 0.51
Area (mm^2): 12.80 +/- 1.54
    
